# Preparing Dataset - How Typhoon impacting flight cancellation

![image](https://drive.google.com/uc?export=view&id=1_b6smorZH3lmG__yrRWtrHiXM9iVmhvn)

In 2025, Hong Kong experienced two powerful Signal-10 typhoons and a record-breaking 12 typhoon signals, the most in its history, according to the Hong Kong Observatory (HKO). These typhoons caused many flight cancellations at Hong Kong International Airport. ([Ref from HKO](https://www.hko.gov.hk/tc/%E5%A4%A9%E6%B0%A3%E9%9A%A8%E7%AD%86/109876/%E7%A0%B4%E7%B4%80%E9%8C%84%E7%9A%842025%E5%B9%B4%E9%A2%A8%E5%AD%A3))

Suppose you are a very smart junior data scientist in Hong Kong International Airport, tasked with gathering preliminary data on the Typhoon data with flight cancellation records. You will collect data from multiple online sources, clean it, and combine it to **create a foundational dataset** for future analysis.


# Part 1: Find out all typhoons with warning signals

Now you are going to develop a **FUNCTION** `get_typhoon_signal(datestart, dateend)` whereas the output could be in list of dictionaries, or pandas dataframe.

Most trivial way should be from [HKO Query Platform of Tropical Cyclone Warning Signals](https://www.hko.gov.hk/en/wxinfo/climat/warndb/warndb1.shtml):
![image](https://drive.google.com/uc?export=view&id=167o2Ls7aMYhVmx-HYrwoc5vFcfdPz5py)
But this function requested has NO restriction on which types of signals.

The underlying dataset could be imported from here: <br>
https://www.hko.gov.hk/dps/wxinfo/climat/warndb/tc.dat

![image](https://drive.google.com/uc?export=view&id=18zCUCbQsCAfqZ35LpqjHqHwjPdzFGY_e)

However, the dataset is quite raw and without column names. <br>
You are recommended to build the following sub-functions:

1. `get_intensity(INTEN)`: Transform those intensity short form to long form. <br>
Please refer to [HKO - Tropical Cyclone Classification](https://www.hko.gov.hk/en/education/tropical-cyclone/classification-naming-characteristics/00145-tropical-cyclone-classification.html)
1. `get_signal(number, direction)`: Transform those directions with 'X' to empty strings, such as  ('8', 'NW') --> '8 NW', ('10', 'X') --> '10'.
1. `get_datetime(year, month, day, hhmm)`: Combine those strings datetime units into datetime values.
1. Any other functions you would like to add.

for beautifying as the output of the platform:

```python
[{'Intensity': 'Super Typhoon', 'Name': 'RAGASA', 'Signal': '8 NW', 'StartTime': '2025-09-23 14:20:00', 'EndTime': '2025-09-24 01:40:00', 'Duration': '11:20'}
, {'Intensity': 'Super Typhoon', 'Name': 'RAGASA', 'Signal': '9', 'StartTime': '2025-09-24 01:40:00', 'EndTime': '2025-09-24 02:40:00', 'Duration': '01:00'}
, {'Intensity': 'Super Typhoon', 'Name': 'RAGASA', 'Signal': '10', 'StartTime': '2025-09-24 02:40:00', 'EndTime': '2025-09-24 13:20:00', 'Duration': '10:40'}
, ...]
```
The data type could be datetime, or string.

Finally please make sure:
* include try-except checking steps for ensuring well-functional.
* include lines of comments detailing how the lines of code working with the format.



In [54]:
import urllib.request, urllib.parse, urllib.error
import json
import ssl
import pandas as pd
from datetime import datetime, timedelta

# Dictionary mapping intensity short forms to their full descriptions
intensity_dict = {
    'TD': 'Tropical Depression',
    'TS': 'Tropical Storm',
    'STS': 'Severe Tropical Storm',
    'T': 'Typhoon',
    'ST': 'Severe Typhoon',
    'SuperT': 'Super Typhoon'
}

def get_intensity(INTEN):
    """
    Convert a short intensity code for typhoons into its full descriptive name.

    Args:
        INTEN (str): The short intensity code (e.g., 'TD' for Tropical Depression, 'TS' for Tropical Storm).

    Returns:
        str: The full description of the intensity. Returns an empty string if the code is not recognized.
    """
    try:
        return intensity_dict[INTEN]  # Return the corresponding full intensity description
    except KeyError:
        return ''  # Return an empty string if the intensity code is not valid

def get_signal(number, direction):
    """
    Format the typhoon warning signal number along with its wind direction for better readability.

    Args:
        number (str): The warning signal number (e.g., '8', '9', etc.).
        direction (str): The wind direction (e.g., 'NW', 'SW') or 'X' if no direction is applicable.

    Returns:
        str: A formatted string combining the signal number and direction, or an empty string if an error occurs.
    """
    try:
        if direction == 'X' or number == '0':  # Return only the number if there's no direction
            return str(number)
        else:
            return f'{number} {direction}'  # Combine number and direction for output
    except Exception:
        return ''  # Return an empty string if there was an error

def get_datetime(year, month, day, hhmm):
    """
    Combine individual date and time components into a datetime object.
    Includes special handling for '2400' hours, converting it to the next day at '0000'.

    Args:
        year (str): The year as a string.
        month (str): The month as a string (1-12).
        day (str): The day as a string (1-31).
        hhmm (str): Time in HHMM format, with '2400' treated as '0000' of the next day.

    Returns:
        datetime: A datetime object representing the combined date and time, or None if parsing fails.
    """
    try:
        if hhmm.zfill(4) == '2400':  # Handle the special case of '2400' hours
            return datetime.strptime(f'{year.zfill(4)}{month.zfill(2)}{day.zfill(2)}0000', "%Y%m%d%H%M") + timedelta(days=1)
        return datetime.strptime(f'{year.zfill(4)}{month.zfill(2)}{day.zfill(2)}{hhmm.zfill(4)}', "%Y%m%d%H%M")
    except ValueError:
        return None  # Return None if the datetime conversion fails


def get_typhoon_signal(datestart, dateend):
    """
    Retrieve typhoon warning signals data from the HKO database within a specified date range.

    Args:
        datestart (str): Start date in YYYYMMDD format
        dateend (str): End date in YYYYMMDD format

    Returns:
        pandas.DataFrame: DataFrame containing typhoon signal information with specified columns.
                          Returns an empty DataFrame on error.
    """
    # Validate the start date format and convert to datetime
    try:
        datestart_dt = datetime.strptime(datestart + '0000', "%Y%m%d%H%M")
    except ValueError:
        return pd.DataFrame()  # Return an empty DataFrame for invalid start date

    # Validate the end date format and convert to datetime
    try:
        dateend_dt = datetime.strptime(dateend + '2359', "%Y%m%d%H%M")
    except ValueError:
        return pd.DataFrame()  # Return an empty DataFrame for invalid end date

    # Attempt to retrieve data from the HKO database
    try:
        url = 'https://www.hko.gov.hk/dps/wxinfo/climat/warndb/tc.dat'
        req = urllib.request.Request(url)
        resp = urllib.request.urlopen(req)
        content = resp.read().decode("utf-8-sig")
    except Exception:
        return pd.DataFrame()  # Return an empty DataFrame if data retrieval fails

    allTyphoon = []  # List to store all typhoon information
    records = content.split('\n')  # Split the data into individual records


    # Process each record to extract relevant typhoon information
    for record in records:
        try:
            parts = record.split('\t')  # Split each record into its components
            duration_st= parts[15]
            duration_hh = int(duration_st[:-2])  # Extract hours
            duration_mm = int(duration_st[-2:])  # Extract hours
            typhoon = {
                'Intensity': get_intensity(parts[1]),
                'Name': parts[2],
                'Signal': get_signal(parts[3], parts[4]),
                'StartTime': get_datetime(parts[8], parts[7], parts[6], parts[5]),
                'EndTime': get_datetime(parts[13], parts[12], parts[11], parts[10]),
                'Duration': f"{duration_hh:02}:{duration_mm:02}"
            }
            if typhoon['Name'] != 'X':  # Only include records with a valid name
                allTyphoon.append(typhoon)  # Add the typhoon information to the list
        except Exception:
            continue  # Skip any record that raises an error

    # Convert the list of typhoon information into a pandas DataFrame
    df = pd.DataFrame(allTyphoon, columns=['Intensity', 'Name', 'Signal', 'StartTime', 'EndTime', 'Duration'])

    # Filter the DataFrame for records within the specified date range and reset the index
    return df[(datestart_dt <= df['StartTime']) & (df['EndTime'] <= dateend_dt)].reset_index(drop=True)

# Example usage: Retrieve typhoon signals between specified dates
allTyphoon = get_typhoon_signal('20250923', '20250924')

# Display the resulting DataFrame
allTyphoon

,Intensity,Name,Signal,StartTime,EndTime,Duration
0,Super Typhoon,RAGASA,8 NW,2025-09-23 14:20:00,2025-09-24 01:40:00,11:20
1,Super Typhoon,RAGASA,9,2025-09-24 01:40:00,2025-09-24 02:40:00,01:00
2,Super Typhoon,RAGASA,10,2025-09-24 02:40:00,2025-09-24 13:20:00,10:40
3,Super Typhoon,RAGASA,8 SE,2025-09-24 13:20:00,2025-09-24 20:20:00,07:00


# Part 2: Getting Wind Speed at Airport

You are going to get the wind speed during typhoon signals for every moment. <br>
In this part, maybe you would involve large size of data. Please start to investigate RAGASA at first, whereas the dates:
* Start Date: 2025-09-22
* End Date: 2025-09-25

You are recommended to head the API: "Regional weather in Hong Kong – the latest 10-minute mean wind direction and wind speed and maximum gust". <br>
Suppose you search data.gov.hk you will get this: https://data.gov.hk/en-data/dataset/hk-hko-rss-latest-ten-minute-wind-info

...

But just the realtime numbers only.

If you search the bottom of the page, we can search historical data.
![wind_speed_data_hist.png](https://drive.google.com/uc?export=view&id=1gx4csrXKjGvXFrOSfBjc9TVO46D68Apb)

But the time parameter in URLs seems not regular.
![wind_speed_url.png](https://drive.google.com/uc?export=view&id=1zJYEGnwCd6eI6ZsHXVzo8A7uicD2qyDm)

We will find those timestamp at first; <br>
then API those data into your colab.

### Part 2.1: Get timestamp of the historical data

Please refer to "Historical Archive File Version API" in [data.gov.hk API Spec - "API for Retrieval of Historical Data"](https://data.gov.hk/en/help/api-spec#api-for-retrieval-of-historical-data), <br>

With the following parameter:
- URL: https://data.weather.gov.hk/weatherAPI/hko_data/regional-weather/latest_10min_wind.csv
- Start Date in format yyyymmdd
- End Date in format yyyymmdd

write a **FUNCTION** `get_wind_speed_url(start_yyyymmdd, end_yyyymmdd)` with returning a **list of datetime strings** in format yyyymmdd-hhmm.<br>

If you try the following parameter:
- start_yyyymmdd: 20250922
- end_yyyymmdd: 20250925

Total no. of yyyymmdd-hhmm = 673 will be returned.

Finally please make sure:
* include try-except checking steps for ensuring well-functional.
* include lines of comments detailing how the lines of code working with the format.

In [55]:
import urllib.request
import urllib.parse
import json
import ssl

# Function to get wind speed timestamps from a historical weather data API
def get_wind_speed_url(start_yyyymmdd, end_yyyymmdd):
    """
    Retrieves wind speed timestamps within a specified date range.

    Parameters:
    - start_yyyymmdd (str): The start date in 'YYYYMMDD' format.
    - end_yyyymmdd (str): The end date in 'YYYYMMDD' format.

    Returns:
    - list: A list of timestamps (str) representing the wind speed data.
            Returns an empty list in case of an error.
    """

    # Ignore SSL certificate errors for the API request
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE

    # Set the service URL for the historical archive
    serviceurl = 'https://app.data.gov.hk/v1/historical-archive/list-file-versions'

    # Define parameters for the API request
    params = {
        'url': 'https://data.weather.gov.hk/weatherAPI/hko_data/regional-weather/latest_10min_wind.csv',
        'start': start_yyyymmdd,
        'end': end_yyyymmdd
    }

    # Construct the full URL with parameters
    url = serviceurl + '?' + urllib.parse.urlencode(params)

    try:
        # Open the URL and read the response
        resp = urllib.request.urlopen(url)
        data = resp.read().decode()

        # Load the JSON response
        js = json.loads(data)

        # Extract timestamps from the JSON response
        timestamps = [ts for ts in js['timestamps']]
        return timestamps

    except urllib.error.HTTPError as e:
        # Handle HTTP errors (e.g., 404, 500)
        print(f"HTTP error occurred: {e.code} - {e.reason}")
        return []

    except urllib.error.URLError as e:
        # Handle URL errors (e.g., network issues)
        print(f"URL error occurred: {e.reason}")
        return []

    except json.JSONDecodeError as e:
        # Handle JSON decoding errors
        print(f"JSON decoding error: {e.msg}")
        return []

    except Exception as e:
        # Catch-all for any other exceptions
        print(f"An unexpected error occurred: {e}")
        return []

# Example usage
# Define the date range for which to retrieve wind speed timestamps
start_yyyymmdd = '20250921'
end_yyyymmdd = '20250925'
get_wind_speed_url(start_yyyymmdd, end_yyyymmdd)



['20250921-0010',
 '20250921-0020',
 '20250921-0029',
 '20250921-0039',
 '20250921-0050',
 '20250921-0059',
 '20250921-0100',
 '20250921-0110',
 '20250921-0119',
 '20250921-0130',
 '20250921-0140',
 '20250921-0150',
 '20250921-0200',
 '20250921-0210',
 '20250921-0220',
 '20250921-0230',
 '20250921-0239',
 '20250921-0249',
 '20250921-0259',
 '20250921-0300',
 '20250921-0310',
 '20250921-0319',
 '20250921-0330',
 '20250921-0340',
 '20250921-0349',
 '20250921-0400',
 '20250921-0410',
 '20250921-0411',
 '20250921-0420',
 '20250921-0429',
 '20250921-0439',
 '20250921-0449',
 '20250921-0450',
 '20250921-0459',
 '20250921-0510',
 '20250921-0519',
 '20250921-0529',
 '20250921-0539',
 '20250921-0549',
 '20250921-0559',
 '20250921-0610',
 '20250921-0620',
 '20250921-0629',
 '20250921-0639',
 '20250921-0650',
 '20250921-0659',
 '20250921-0700',
 '20250921-0709',
 '20250921-0710',
 '20250921-0719',
 '20250921-0729',
 '20250921-0739',
 '20250921-0740',
 '20250921-0750',
 '20250921-0759',
 '20250921

### Part 2.2: Get Data and save as a dataframe from the timestamp

Again, please refer to "Historical Archive File Download API" in [data.gov.hk API Spec - "API for Retrieval of Historical Data"](https://data.gov.hk/en/help/api-spec#api-for-retrieval-of-historical-data).

With the following parameter:
- URL: https://data.weather.gov.hk/weatherAPI/hko_data/regional-weather/latest_10min_wind.csv
- time: output as Part 2.1

write a **FUNCTION** `get_wind_speed_airport(start_datetime, end_datetime)` with returning a dataFrame.

Notes about your coding as the following:
- Each response is in bytes output. You need to [decode](https://www.geeksforgeeks.org/python-strings-decode-method/) before pre-processing it.
- Each response is in csv form. You need to use [StringIO](https://sparkbyexamples.com/pandas/how-to-read-csv-from-string-in-pandas/) for reading string into dataframe.
- Please drop those duplicates before concluding the dataframe. Seems some csv are identical within some timestamps
- [The most important] Please extract the row with "Automatic Weather Station" of "Chek Lap Kok" ONLY.
- The `get_wind_speed_airport` function please utilize the function in part 2.1.
- Please expect the scrapping costing at around 15 mins, for the case of RAGASA.

![df_wind_speed.png](https://drive.google.com/uc?export=view&id=1SCHl-R0D-q0hXMhvCwdq1W3M9PSXvnJe)

Finally please make sure:
* include try-except checking steps for ensuring well-functional.
* include lines of comments detailing how the lines of code working with the format.

In [56]:
import urllib.request
import urllib.parse
import pandas as pd
from io import StringIO
import ssl
from datetime import datetime
from typing import Optional

def get_wind_speed_airport(start_datetime: str, end_datetime: str) -> Optional[pd.DataFrame]:
    """
    Fetch wind speed data for the Chek Lap Kok airport within the specified datetime range.

    Parameters:
    - start_datetime: Start date and time in 'yyyymmdd-hhmm' format.
    - end_datetime: End date and time in 'yyyymmdd-hhmm' format.

    Returns:
    - DataFrame containing wind speed data or None if no data is collected.
    """
    # Extract the date part for get_wind_speed_url
    start_yyyymmdd = start_datetime[:8]
    end_yyyymmdd = end_datetime[:8]

    # Create a context to handle SSL certificates
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE

    serviceurl = 'https://app.data.gov.hk/v1/historical-archive/get-file'
    all_data = []  # List to store DataFrames for each timestamp

    # Get timestamps in 'yyyymmdd-hhmm' format
    timestamps = get_wind_speed_url(start_yyyymmdd, end_yyyymmdd)

    timestamp_dt_list = []
    for ts in timestamps:
        try:
            # Convert string timestamp to datetime object
            dt = datetime.strptime(ts, '%Y%m%d-%H%M')
            timestamp_dt_list.append(dt)
        except ValueError as e:
            print(f"Error parsing timestamp '{ts}': {e}")

    # Convert start_datetime and end_datetime to datetime objects for comparison
    try:
        start_dt = datetime.strptime(start_datetime, '%Y%m%d-%H%M')
        end_dt = datetime.strptime(end_datetime, '%Y%m%d-%H%M')
    except ValueError as e:
        print(f"Error parsing start or end datetime: {e}")
        return None

    for timestamp in timestamp_dt_list:
        # Continue if timestamp is not within the desired range
        if not (start_dt <= timestamp <= end_dt):
            continue
        timestamp_str = timestamp.strftime('%Y%m%d-%H%M')

        # Construct the API request parameters
        params = {
            'url': 'https://data.weather.gov.hk/weatherAPI/hko_data/regional-weather/latest_10min_wind.csv',
            'time': timestamp_str
        }
        url = serviceurl + '?' + urllib.parse.urlencode(params)

        try:
            # Fetch the data from the constructed URL
            resp = urllib.request.urlopen(url, context=ctx)
            data = resp.read().decode()  # Decode the byte response

            # Use StringIO to read the CSV data into a DataFrame
            df = pd.read_csv(StringIO(data))

            # Filter for "Chek Lap Kok" and drop duplicates
            df_chek_lap_kok = df[df['Automatic Weather Station'] == 'Chek Lap Kok'].drop_duplicates()

            if not df_chek_lap_kok.empty:
                all_data.append(df_chek_lap_kok)

        except urllib.error.HTTPError as http_err:
            print(f"HTTP error occurred for {timestamp.strftime('%Y%m%d')}: {http_err}")
        except ValueError as e:
            print(f"ValueError while processing data for {timestamp.strftime('%Y%m%d')}: {e}")
        except Exception as e:
            print(f"Error fetching data for {timestamp.strftime('%Y%m%d')}: {e}")

    try:
        # Concatenate the DataFrames in the list to one large DataFrame
        df_all = pd.concat(all_data, ignore_index=True)

        # Filter out rows that have the same datetime, choosing the first row of data
        df_all = df_all.groupby(by=['Date time', 'Automatic Weather Station']).first().reset_index()

        # Convert 'Date time' from string to datetime
        if 'Date time' in df_all.columns:
            df_all['Date time'] = pd.to_datetime(df_all['Date time'], format='%Y%m%d%H%M')

        return df_all
    except Exception as e:
        print(f"Error during concatenation or processing: {e}")
        # Return an empty DataFrame with specified columns on failure
        return pd.DataFrame(columns=['Date time', 'Automatic Weather Station',
                                     '10-Minute Mean Wind Direction(Compass points)',
                                     '10-Minute Mean Speed(km/hour)',
                                     '10-Minute Maximum Gust(km/hour)'])

# Example usage
get_wind_speed_airport('20250922-2300', '20250922-2359')

,Date time,Automatic Weather Station,10-Minute Mean Wind Direction(Compass points),10-Minute Mean Speed(km/hour),10-Minute Maximum Gust(km/hour)
0,2025-09-22 23:00:00,Chek Lap Kok,North,18,25.0
1,2025-09-22 23:10:00,Chek Lap Kok,North,17,23.0
2,2025-09-22 23:20:00,Chek Lap Kok,North,22,29.0
3,2025-09-22 23:30:00,Chek Lap Kok,North,19,25.0
4,2025-09-22 23:40:00,Chek Lap Kok,North,19,25.0


# Part 3: Getting Status - Delays or Cancels of the scheduled flights

Getting the Flight schedule information of Hong Kong International Airport (HKIA), you can use the API from HKAA: <br>
https://data.gov.hk/en-data/dataset/aahk-team1-flight-info

Please refer to the [API specification](https://www.hongkongairport.com/iwov-resources/misc/opendata/Flight_Information_DataSpec_en.pdf), with specific parameters
then write a **FUNCTION** `get_flight_status(start_date, end_date)` to obtain a dataset as the capscreen shown through [HKIA official page](https://www.hongkongairport.com/en/flights/departures/passenger.page):

![wind_speed_url.png](https://drive.google.com/uc?export=view&id=1M2arAXxj5UXkRgd5NyPXnynqGeJdm9l8)

Note:
* Since the function input is a date range, you need to use for-loop for obtaining a dataset for every single day.
* We consider scheduled flights only - from Hong Kong, and for passengers.
* Make sure each row for a flight only - take care if the flight has shared codes, and multiple destinations. Take the first flight no., and the last destination if necessary.
* Some columns such as "Transfer Desk" are not in the API - use NULL or drop the column as you like.

Finally please make sure:
* include try-except checking steps for ensuring well-functional.
* include lines of comments detailing how the lines of code working with the format.

In [57]:
import urllib.request
import urllib.parse
import urllib.error
import json
import ssl
import pandas as pd
from datetime import datetime, timedelta

def get_single_day_flight_data(date, arrival='false', cargo='false', language='en'):
    """
    Retrieve flight data for a specific day from the Hong Kong Airport API.

    Args:
      date (str): The date for which to retrieve flight data, formatted as 'YYYY-MM-DD'.
      arrival (str): Specify 'true' to fetch arrival flights or 'false' for departure flights.
      cargo (str): Specify 'true' to include cargo flights or 'false' for passenger flights.
      language (str): The language code for the response, default is 'en' for English.

    Returns:
        dict: A JSON object containing the flight data for the specified date.
          Returns an empty dictionary if an error occurs during data retrieval.

    Note:
    This function sends requests with headers that mimic browser behavior to prevent blocking by the server.
    """

    url = f'https://www.hongkongairport.com/flightinfo-rest/rest/flights/past?date={date}&arrival={arrival}&cargo={cargo}&lang={language}'

    # Ignore SSL certificate errors for secure connection
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE

    req = urllib.request.Request(
        url,
        headers={
            "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                           "AppleWebKit/537.36 (KHTML, like Gecko) "
                           "Chrome/124.0.0.0 Safari/537.36"),
            "Accept": "application/json, text/plain, */*",
            "Referer": "https://www.hongkongairport.com/",
            "Accept-Language": "en-US,en;q=0.9",
        }
    )

    try:
        # Attempt to open the URL and read the response
        with urllib.request.urlopen(req, context=ctx, timeout=20) as resp:
            data = resp.read().decode("utf-8")
            return json.loads(data)  # Parse and return the JSON data
    except Exception as e:
        # Handle any exceptions that occur during the request
        return {}  # Return an empty dictionary if the request fails

def get_flight_status(start_date, end_date):
    """
    Fetches flight status information for scheduled passenger departure flights at Hong Kong Airport over a specified date range.

    Parameters:
      start_date (str): The starting date for data retrieval, formatted as 'YYYY-MM-DD'.
      end_date (str): The ending date for data retrieval, formatted as 'YYYY-MM-DD'.

    Returns:
      pandas.DataFrame: A DataFrame containing flight details with the following columns:
                      ['DATE', 'TIME', 'AIRLINE', 'FLIGHT', 'DESTINATION', 'TERMINAL', 'CHECK-IN', 'GATE', 'STATUS'].
                      If an error occurs, an empty DataFrame is returned.

    Additional Notes:
      - For flights with shared codes, only the first flight number is retained.
      - If multiple destinations are available, the last destination is selected.
      - Duplicate flight records are removed to ensure unique entries.
    """

    try:
        # Validate start date format
        start_date_dt = datetime.strptime(start_date, "%Y-%m-%d")
    except ValueError:
        return []  # Return an empty list if format is incorrect

    try:
        # Validate end date format
        end_date_dt = datetime.strptime(end_date, "%Y-%m-%d")
    except ValueError:
        return []  # Return an empty list if format is incorrect

    # Generate a range of dates from start to end
    date_range = pd.date_range(start_date_dt, end_date_dt)
    raw_data_list = []

    for date in date_range:
        try:
            # Fetch flight data for each date in the range
            raw_data_list.append(get_single_day_flight_data(date.strftime('%Y-%m-%d'), 'false', 'false', 'en'))
        except Exception:
            continue  # Skip if an error occurs while fetching data

    flight_list = []  # List to store processed flight data

    for dayFlightData in raw_data_list:  # Process each day's flight data
        try:
            processingDate = dayFlightData[0]['date']
            datas = dayFlightData[0]['list']
            for row in datas:  # Process each flight record for the day
                try:
                    flight_info = row['flight'][0]
                    flight_list.append({
                        'DATE': processingDate,
                        'TIME': row['time'],
                        'AIRLINE': flight_info['airline'],  # First airline in the list
                        'FLIGHT': flight_info['no'],  # First flight number in the list
                        'DESTINATION': row['destination'][-1],  # Last destination in the list
                        'TERMINAL': row['terminal'],
                        'CHECK-IN': '/'.join(row['aisle']),  # Join check-in locations
                        'GATE': row['gate'],
                        'STATUS': row['status']  # Flight status
                    })  # Append flight data to the list
                except Exception:
                    continue  # Skip flight record if an error occurs
        except Exception:
            continue  # Skip day if an error occurs while processing data

    # Return a DataFrame of unique flight records
    return pd.DataFrame(flight_list, columns=['DATE', 'TIME', 'AIRLINE', 'FLIGHT', 'DESTINATION', 'TERMINAL', 'CHECK-IN', 'GATE', 'STATUS']).drop_duplicates()

# Retrieve flight data for the specified date range
df = get_flight_status('2025-09-22', '2025-09-25')
df

,DATE,TIME,AIRLINE,FLIGHT,DESTINATION,TERMINAL,CHECK-IN,GATE,STATUS
0,2025-09-22,00:05,CPA,CX 143,PER,T1,A,36,Dep 00:29
1,2025-09-22,00:05,CPA,CX 261,CDG,T1,A,1,Dep 00:38
2,2025-09-22,00:05,CPA,CX 383,ZRH,T1,A,64,Dep 00:31
3,2025-09-22,00:05,CPA,CX 880,LAX,T1,B/C,41,Dep 00:13
4,2025-09-22,00:20,CPA,CX 105,MEL,T1,A,23,Dep 01:23
...,...,...,...,...,...,...,...,...,...
1610,2025-09-25,23:15,SWR,LX 139,ZRH,T1,F,25,Dep 23:54
1611,2025-09-25,23:25,DLH,LH 797,FRA,T1,F,47,Dep 23:43
1612,2025-09-25,23:40,HKE,UO 624,HND,T1,H,207,Dep 23:34
1613,2025-09-25,23:55,CPA,CX 101,SYD,T1,A,48,Dep 00:01 (26/09/2025)


# Part 4: Merge them together to the goal dataset

Coming to the last mile of the Mid-Term Assessment - write the **MAIN PROGRAM** which merging them together:
* Please produce a dataset
  * [Part 1] for an abitrary typhoon, the date range with any typhoon signal
  * [Part 3] for each row representing a flight
  * [Part 2] with the wind speed, direction, and 10-minutes gust at the flight's schedule time.
* If the flight time does not match the wind-speed timestamp, get the wind speed with latest timestamp.

Please try to produce for the typhoons "TAPAH 塔巴", "RAGASA 樺加沙", and "MATMO 麥德姆".

In [58]:
import urllib.request, urllib.parse, urllib.error
import json
import ssl
import pandas as pd
from datetime import datetime, timedelta
import bisect

def get_flight_cancelled_due_to_typhoon_dataset(datestart=None, dateend=None, typhoonName=None):
    """
    This function constructs a detailed dataset that combines information from typhoon signals, flight records, and wind speed data.

    Parameters:
        datestart (str, optional): The start date in YYYYMMDD format. If omitted, the function will filter by typhoonName.
        dateend (str, optional): The end date in YYYYMMDD format. If omitted, the function will filter by typhoonName.
        typhoonName (str, optional): The name of a specific typhoon for filtering. If not provided, the date range will be utilized.

    Returns:
        pandas.DataFrame: A dataset that includes:
                          - Details of typhoon signals (including intensity, name, type, and timing)
                          - Information about flights (date, time, airline, flight number, destination, and status)
                          - Wind speed metrics (direction, mean speed, maximum gust) at the time of the flights.
                          An empty DataFrame will be returned if an error occurs.

    Workflow:
        1. Checks the validity of input parameters and retrieves typhoon signal data.
        2. Gathers flight information for the periods influenced by typhoon signals.
        3. Acquires corresponding wind speed data, allowing for a time buffer.
        4. Integrates all datasets using time-based joins.
        5. Outputs the final dataset.

    Example Usage:
        >>> df = get_flight_cancelled_due_to_typhoon_dataset(typhoonName='RAGASA')
        >>> print(df.shape)  # Displays the number of flights affected by the RAGASA typhoon.
    """
    # Validate input parameters and return an empty DataFrame if conditions are not met
    if (not (datestart is not None and dateend is not None)) and typhoonName is None:
        return pd.DataFrame()  # Return empty DataFrame if no inputs are provided

    # Check if datestart and dateend are provided and valid
    if datestart is not None and dateend is not None:
        try:
            datestart_dt = datetime.strptime(datestart, "%Y%m%d")  # Convert start date string to datetime
        except ValueError:
            return pd.DataFrame()  # Return empty DataFrame for invalid datestart

        try:
            dateend_dt = datetime.strptime(dateend, "%Y%m%d")  # Convert end date string to datetime
        except ValueError:
            return pd.DataFrame()  # Return empty DataFrame for invalid dateend

    # Fetch typhoon data based on provided dates or defaults
    try:
        if datestart and dateend:
            typhoonData = get_typhoon_signal(datestart, dateend)  # Retrieve typhoon signal data within date range
        else:
            typhoonData = get_typhoon_signal('19460101', datetime.now().strftime('%Y%m%d'))  # Default date range

        if typhoonName is not None:
            typhoonData = typhoonData[typhoonData['Name'] == typhoonName].reset_index(drop=True)  # Filter by typhoon name

        if len(typhoonData) == 0:
            return pd.DataFrame()  # Return empty DataFrame if no typhoons found
    except Exception as e:  # General error handling for typhoon data retrieval
        return pd.DataFrame()

    # Get flight data that falls under typhoon signals
    typhoonGroupdata = typhoonData.groupby(by=['Name']).agg({'StartTime': 'min', 'EndTime': 'max'}).reset_index()  # Group by typhoon name to get start and end times
    flight_data_list = []
    try:
        for _, row in typhoonGroupdata.iterrows():
            flight_data_list.append(get_flight_status(row['StartTime'].strftime('%Y-%m-%d'), row['EndTime'].strftime('%Y-%m-%d')))  # Retrieve flight data for each typhoon period
        flight_data_raw = pd.concat(flight_data_list)  # Concatenate all flight data into a single DataFrame
        # Create a timestamp column for easier filtering and joining later
        flight_data_raw['flight_ts'] = flight_data_raw.apply(lambda x: datetime.strptime(f"{x['DATE']}{x['TIME']}", '%Y-%m-%d%H:%M'), axis=1)
    except Exception as e:  # Error handling for flight data retrieval
        return pd.DataFrame()  # Return empty DataFrame if flight data retrieval fails

    # Merge typhoon data with flight data based on timestamps
    try:
        typhoon_flight_merged_conditional = typhoonData.merge(flight_data_raw, how='cross')  # Perform a cross join to get all combinations
        filtered_flight_data =  typhoon_flight_merged_conditional[
            (typhoon_flight_merged_conditional['flight_ts'] >= typhoon_flight_merged_conditional['StartTime']) &
            (typhoon_flight_merged_conditional['flight_ts'] <= typhoon_flight_merged_conditional['EndTime']
        )].drop_duplicates(subset=['DATE', 'TIME', 'AIRLINE', 'FLIGHT', 'DESTINATION', 'TERMINAL', 'CHECK-IN', 'GATE', 'STATUS', 'flight_ts'], keep='last')  # Filter flights within typhoon periods and retain unique entries
        del typhoon_flight_merged_conditional  # Clean up by deleting the merged DataFrame
    except Exception as e:  # Error handling for merging flight and typhoon data
        return pd.DataFrame()  # Return empty DataFrame if merging fails

    # Retrieve wind speed data
    wind_speed_data_raw_list = []
    try:
        for _, row in typhoonGroupdata.iterrows():
            try:
                # Create time range for wind speed data retrieval
                start_time_bu = pd.to_datetime(row['StartTime']) - timedelta(minutes=30)  # Start time with a buffer
                end_time_bu = pd.to_datetime(row['EndTime']) + timedelta(minutes=30)  # End time with a buffer

                # Format times for wind speed data query
                start_time_bu_str = start_time_bu.strftime('%Y%m%d-%H%M')
                end_time_bu_str = end_time_bu.strftime('%Y%m%d-%H%M')

                # Append wind speed data to the list
                wind_speed_data_raw_list.append(get_wind_speed_airport(start_time_bu_str, end_time_bu_str))
            except Exception:
                continue  # Skip any errors in wind speed data retrieval
        wind_speed_data = pd.concat(wind_speed_data_raw_list).drop_duplicates()  # Concatenate wind speed data and remove duplicates
        wind_speed_data['Date time timestamp'] = wind_speed_data['Date time']  # Create a timestamp column for merging
    except Exception as e:  # Error handling for wind speed data retrieval
        return pd.DataFrame()  # Return empty DataFrame if wind speed data retrieval fails

    # Merge filtered flight data with wind speed data
    try:
        res = pd.merge_asof(filtered_flight_data, wind_speed_data, left_on='flight_ts', right_on='Date time timestamp')  # Perform an asof merge based on flight timestamps
        res = res.drop(columns=['Duration', 'flight_ts', 'Date time timestamp', 'Date time'])  # Drop unnecessary columns from the result
    except Exception as e:  # Error handling for merging flight data with wind speed data
        return pd.DataFrame()  # Return empty DataFrame if merging fails

    return res  # Return the final merged DataFrame

In [62]:
get_flight_cancelled_due_to_typhoon_dataset(datestart = None, dateend = None, typhoonName = 'TAPAH')

,Intensity,Name,Signal,StartTime,EndTime,DATE,TIME,AIRLINE,FLIGHT,DESTINATION,TERMINAL,CHECK-IN,GATE,STATUS,Automatic Weather Station,10-Minute Mean Wind Direction(Compass points),10-Minute Mean Speed(km/hour),10-Minute Maximum Gust(km/hour)
0,Severe Tropical Storm,TAPAH,1,2025-09-05 22:20:00,2025-09-07 02:40:00,2025-09-05,22:20,AFR,AF 185,CDG,T1,D,64,Dep 22:19,Chek Lap Kok,East,14.0,21.0
1,Severe Tropical Storm,TAPAH,1,2025-09-05 22:20:00,2025-09-07 02:40:00,2025-09-05,22:25,CRK,HX 017,SYD,T1,K,218,Dep 22:30,Chek Lap Kok,East,14.0,21.0
2,Severe Tropical Storm,TAPAH,1,2025-09-05 22:20:00,2025-09-07 02:40:00,2025-09-05,22:30,CEB,5J 121,CRK,T1,L,16,Dep 22:21,Chek Lap Kok,East,12.0,17.0
3,Severe Tropical Storm,TAPAH,1,2025-09-05 22:20:00,2025-09-07 02:40:00,2025-09-05,22:30,THY,TK 071,IST,T1,D,49,Dep 22:35,Chek Lap Kok,East,12.0,17.0
4,Severe Tropical Storm,TAPAH,1,2025-09-05 22:20:00,2025-09-07 02:40:00,2025-09-05,22:30,UAL,UA 878,SFO,T1,G,45,Dep 22:21,Chek Lap Kok,East,12.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274,Severe Tropical Storm,TAPAH,1,2025-09-08 19:40:00,2025-09-09 04:20:00,2025-09-09,02:20,CRK,HX 707,DPS,T1,K,214,Dep 02:15,Chek Lap Kok,South,28.0,39.0
1275,Severe Tropical Storm,TAPAH,1,2025-09-08 19:40:00,2025-09-09 04:20:00,2025-09-09,02:20,HKE,UO 658,CJJ,T1,H,,Cancelled,Chek Lap Kok,South,28.0,39.0
1276,Severe Tropical Storm,TAPAH,1,2025-09-08 19:40:00,2025-09-09 04:20:00,2025-09-09,02:20,HKE,UO 676,CJU,T1,H,213,Dep 02:15,Chek Lap Kok,South,28.0,39.0
1277,Severe Tropical Storm,TAPAH,1,2025-09-08 19:40:00,2025-09-09 04:20:00,2025-09-09,02:35,CRK,HX 606,NRT,T1,K,207,Dep 02:41,Chek Lap Kok,South,29.0,47.0


In [64]:
get_flight_cancelled_due_to_typhoon_dataset(datestart = None, dateend = None, typhoonName = 'RAGASA')

,Intensity,Name,Signal,StartTime,EndTime,DATE,TIME,AIRLINE,FLIGHT,DESTINATION,TERMINAL,CHECK-IN,GATE,STATUS,Automatic Weather Station,10-Minute Mean Wind Direction(Compass points),10-Minute Mean Speed(km/hour),10-Minute Maximum Gust(km/hour)
0,Super Typhoon,RAGASA,1,2025-09-22 12:20:00,2025-09-22 21:40:00,2025-09-22,12:20,CES,MU 6098,LHW,T1,J,18,Dep 12:14,Chek Lap Kok,Northwest,19.0,22.0
1,Super Typhoon,RAGASA,1,2025-09-22 12:20:00,2025-09-22 21:40:00,2025-09-22,12:20,CRK,HX 2350,DNH,T1,K,208,Dep 12:17,Chek Lap Kok,Northwest,19.0,22.0
2,Super Typhoon,RAGASA,1,2025-09-22 12:20:00,2025-09-22 21:40:00,2025-09-22,12:20,KME,KR 508,KTI,T1,K,41,Dep 12:35,Chek Lap Kok,Northwest,19.0,22.0
3,Super Typhoon,RAGASA,1,2025-09-22 12:20:00,2025-09-22 21:40:00,2025-09-22,12:20,KAL,KE 174,ICN,T1,G,6,Dep 12:31,Chek Lap Kok,Northwest,19.0,22.0
4,Super Typhoon,RAGASA,1,2025-09-22 12:20:00,2025-09-22 21:40:00,2025-09-22,12:25,SIA,SQ 875,SIN,T1,F,7,Dep 12:54,Chek Lap Kok,Northwest,19.0,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,Super Typhoon,RAGASA,1,2025-09-25 08:20:00,2025-09-25 11:20:00,2025-09-25,11:15,CPA,CX 739,SIN,T1,A,27,Dep 11:26,Chek Lap Kok,Southeast,35.0,48.0
1185,Super Typhoon,RAGASA,1,2025-09-25 08:20:00,2025-09-25 11:20:00,2025-09-25,11:15,UAL,UA 862,SFO,,,,Cancelled,Chek Lap Kok,Southeast,35.0,48.0
1186,Super Typhoon,RAGASA,1,2025-09-25 08:20:00,2025-09-25 11:20:00,2025-09-25,11:20,PAL,PR 301,MNL,T1,F,28,Dep 11:08,Chek Lap Kok,Southeast,36.0,51.0
1187,Super Typhoon,RAGASA,1,2025-09-25 08:20:00,2025-09-25 11:20:00,2025-09-25,11:20,HKE,UO 898,KIX,T1,H,203,Dep 11:19,Chek Lap Kok,Southeast,36.0,51.0


In [65]:
get_flight_cancelled_due_to_typhoon_dataset(datestart = None, dateend = None, typhoonName = 'MATMO')

,Intensity,Name,Signal,StartTime,EndTime,DATE,TIME,AIRLINE,FLIGHT,DESTINATION,TERMINAL,CHECK-IN,GATE,STATUS,Automatic Weather Station,10-Minute Mean Wind Direction(Compass points),10-Minute Mean Speed(km/hour),10-Minute Maximum Gust(km/hour)
0,Typhoon,MATMO,1,2025-10-03 19:40:00,2025-10-04 12:20:00,2025-10-03,19:40,EVA,BR 872,TPE,T1,E,30,Dep 19:55,Chek Lap Kok,East,12.0,15.0
1,Typhoon,MATMO,1,2025-10-03 19:40:00,2025-10-04 12:20:00,2025-10-03,19:40,QTR,QR 817,DOH,T1,E,5,Dep 19:53,Chek Lap Kok,East,12.0,15.0
2,Typhoon,MATMO,1,2025-10-03 19:40:00,2025-10-04 12:20:00,2025-10-03,19:45,HKE,UO 226,NGB,T1,H,211,Dep 20:07,Chek Lap Kok,East,12.0,15.0
3,Typhoon,MATMO,1,2025-10-03 19:40:00,2025-10-04 12:20:00,2025-10-03,19:50,VJC,VJ 877,SGN,T1,D,12,Dep 20:05,Chek Lap Kok,Northeast,11.0,14.0
4,Typhoon,MATMO,1,2025-10-03 19:40:00,2025-10-04 12:20:00,2025-10-03,20:00,CPA,CX 988,CAN,T1,A,13,Dep 20:02,Chek Lap Kok,East,12.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937,Typhoon,MATMO,1,2025-10-05 15:40:00,2025-10-05 22:20:00,2025-10-05,21:50,CPA,CX 617,BKK,T1,A,48,Dep 22:00,Chek Lap Kok,East,19.0,27.0
938,Typhoon,MATMO,1,2025-10-05 15:40:00,2025-10-05 22:20:00,2025-10-05,21:55,CPA,CX 939,MNL,T1,B/C,69,Dep 22:33,Chek Lap Kok,East,19.0,27.0
939,Typhoon,MATMO,1,2025-10-05 15:40:00,2025-10-05 22:20:00,2025-10-05,22:05,HKE,UO 710,BKK,T1,H,205,Dep 22:20,Chek Lap Kok,East,19.0,27.0
940,Typhoon,MATMO,1,2025-10-05 15:40:00,2025-10-05 22:20:00,2025-10-05,22:15,HKE,UO 234,PKX,T1,H,201,Dep 22:12,Chek Lap Kok,East,24.0,33.0
